In [ ]:
BATCH_SIZE = 64
MAX_EPOCHS = 10

In [ ]:
!rm -r ./reduce-mode-collapse-in-gan
!git clone -b stacked_mnist_experiment https://github.com/ThViviani/reduce-mode-collapse-in-gan.git

import sys; sys.path.append('./reduce-mode-collapse-in-gan')

rm: cannot remove './reduce-mode-collapse-in-gan': No such file or directory
Cloning into 'reduce-mode-collapse-in-gan'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 267 (delta 8), reused 17 (delta 6), pack-reused 242 (from 1)
Receiving objects: 100% (267/267), 5.38 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [ ]:
!pip install -r ./reduce-mode-collapse-in-gan/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import torch
import torchvision
import torch.utils
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import wandb
import lightning as L

from torch.utils.data import Dataset
from torchvision import transforms, utils, models
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer, seed_everything

In [ ]:
compose = transforms.Compose([
    transforms.Pad(padding=2), # [28, 28] -> [32, 32]
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5], [0.5])
])

train_dataset = torchvision.datasets.MNIST('MNIST/raw/train-images-idx3-ubyte', download=True, train=True, transform=compose)
val_dataset = torchvision.datasets.MNIST('MNIST/raw/train-images-idx3-ubyte', download=True, train=False, transform=compose)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 16053606.83it/s]


Extracting MNIST/raw/train-images-idx3-ubyte/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw/train-images-idx3-ubyte/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 479202.54it/s]


Extracting MNIST/raw/train-images-idx3-ubyte/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw/train-images-idx3-ubyte/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 4455664.86it/s]


Extracting MNIST/raw/train-images-idx3-ubyte/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw/train-images-idx3-ubyte/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 9454356.71it/s]

Extracting MNIST/raw/train-images-idx3-ubyte/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw/train-images-idx3-ubyte/MNIST/raw



In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 92.6MB/s]


# Prepare resnet for transfer learning

In [ ]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(2048, 10, bias=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
if torch.cuda.is_available():
    resnet50 = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
import time
from tqdm.notebook import tqdm

print('Training....')
total = 0
correct = 0
start = time.time()

for epoch in tqdm(range(MAX_EPOCHS)):

    for i, data in enumerate(train_loader, 1):
        images, labels = data

        if(torch.cuda.is_available()):
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        if(i%100 == 0):
            print('Epoch: {} Batch: {} loss: {}'.format(epoch, i, loss.item()))

        loss.backward()
        optimizer.step()

print('Training Completed in: {} secs'.format(time.time()-start))
print('Training accuracy: {} %'.format((correct/total)*100))

Training....


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 Batch: 100 loss: 0.33821535110473633
Epoch: 0 Batch: 200 loss: 0.5689288377761841
Epoch: 0 Batch: 300 loss: 0.21157129108905792
Epoch: 0 Batch: 400 loss: 0.13194940984249115
Epoch: 0 Batch: 500 loss: 0.13959018886089325
Epoch: 0 Batch: 600 loss: 0.06623256206512451
Epoch: 0 Batch: 700 loss: 0.08265786617994308
Epoch: 0 Batch: 800 loss: 0.1211196705698967
Epoch: 0 Batch: 900 loss: 0.1987997144460678
Epoch: 1 Batch: 100 loss: 0.19487138092517853
Epoch: 1 Batch: 200 loss: 0.09376946091651917
Epoch: 1 Batch: 300 loss: 0.10469948500394821
Epoch: 1 Batch: 400 loss: 0.039698339998722076
Epoch: 1 Batch: 500 loss: 0.062339749187231064
Epoch: 1 Batch: 600 loss: 0.03366245701909065
Epoch: 1 Batch: 700 loss: 0.04640873894095421
Epoch: 1 Batch: 800 loss: 0.19963257014751434
Epoch: 1 Batch: 900 loss: 0.054064903408288956
Epoch: 2 Batch: 100 loss: 0.005793439690023661
Epoch: 2 Batch: 200 loss: 0.0985005721449852
Epoch: 2 Batch: 300 loss: 0.01060703955590725
Epoch: 2 Batch: 400 loss: 0.066204

In [ ]:
from sklearn.metrics import classification_report


def compute_metrics(model, loader, targets, device=device):
  preds = []
  for batch in tqdm(val_loader):
    x, _ = batch
    x = x.to(device)
    outputs = model(x)
    outputs = outputs.detach().cpu()
    _, predicted = torch.max(outputs.data, 1)
    preds.extend(predicted)

  print(classification_report(targets, preds))

In [ ]:
compute_metrics(resnet50, val_loader, val_dataset.targets)

  0%|          | 0/157 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.98      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.98      1.00      0.99      1010
           4       0.98      1.00      0.99       982
           5       1.00      0.98      0.99       892
           6       0.99      0.98      0.99       958
           7       0.99      0.96      0.97      1028
           8       0.98      0.99      0.99       974
           9       0.99      0.97      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



# Saving checkpoint

In [ ]:
torch.save(resnet50.state_dict(), 'resnet50_on_mnist_99.pth')

In [ ]:
model_hat = models.resnet50(pretrained=False)

model_hat.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_hat.fc = nn.Linear(2048, 10, bias=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
model_hat.load_state_dict(torch.load('/content/resnet50_on_mnist_99.pth', weights_only=True))

<All keys matched successfully>

In [ ]:
compute_metrics(model_hat, val_loader, val_dataset.targets, device=torch.device('cpu'))

  0%|          | 0/157 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.97      1.00      0.98      1010
           4       0.98      1.00      0.99       982
           5       0.99      0.98      0.99       892
           6       1.00      0.98      0.99       958
           7       0.99      0.97      0.98      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
from torchvision import models


class ResNet50OnMNIST(nn.Module):
  def __init__(self, path_to_checkpoint='/content/resnet50_on_mnist_99.pth'):
    super().__init__()

    self.model = models.resnet50(pretrained=False)
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.model.fc = nn.Linear(2048, 10, bias=True)
    self.model.load_state_dict(torch.load(path_to_checkpoint, weights_only=True))

  def forward(self, x):
    return self.model(x)

  def predict(self, x):
    outputs = self.model(x)
    _, predicted = torch.max(outputs.data, 1)
    return predicted

In [ ]:
model = ResNet50OnMNIST()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
compute_metrics(model, val_loader, val_dataset.targets, device=torch.device('cpu'))

  0%|          | 0/157 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.97      1.00      0.98      1010
           4       0.98      1.00      0.99       982
           5       0.99      0.98      0.99       892
           6       1.00      0.98      0.99       958
           7       0.99      0.97      0.98      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
model.predict(next(iter(val_loader))[0])

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3])